In [ ]:
!unzip -o "MelihDurhasanDSA210SprintermProject-main (1).zip" -d /content/project



Archive:  MelihDurhasanDSA210SprintermProject-main (1).zip
47cc2826821b2e12d00a3995f0a8a7bf87925502
  inflating: /content/project/MelihDurhasanDSA210SprintermProject-main/Crime_Analysis_Final_Findings_Cleaned_Final.pdf  
 extracting: /content/project/MelihDurhasanDSA210SprintermProject-main/DSA210_datas.zip  
  inflating: /content/project/MelihDurhasanDSA210SprintermProject-main/README.md  
  inflating: /content/project/MelihDurhasanDSA210SprintermProject-main/data-statistics.ipynb  


In [ ]:
!unzip -o /content/project/MelihDurhasanDSA210SprintermProject-main/DSA210_datas.zip -d /content/data


Archive:  /content/project/MelihDurhasanDSA210SprintermProject-main/DSA210_datas.zip
  inflating: /content/data/chicago_population_named.csv  
  inflating: /content/data/weather-data.csv  
  inflating: /content/data/crime-data.csv  


In [ ]:
!unzip -o /content/project/MelihDurhasanDSA210SprintermProject-main/DSA210_datas.zip -d /content/data


Archive:  /content/project/MelihDurhasanDSA210SprintermProject-main/DSA210_datas.zip
  inflating: /content/data/chicago_population_named.csv  
  inflating: /content/data/weather-data.csv  
  inflating: /content/data/crime-data.csv  


In [ ]:
!ls /content/project/MelihDurhasanDSA210SprintermProject-main/data


ls: cannot access '/content/project/MelihDurhasanDSA210SprintermProject-main/data': No such file or directory


In [ ]:
!unzip -o /content/project/MelihDurhasanDSA210SprintermProject-main/DSA210_datas.zip -d /content/data


Archive:  /content/project/MelihDurhasanDSA210SprintermProject-main/DSA210_datas.zip
  inflating: /content/data/chicago_population_named.csv  
  inflating: /content/data/weather-data.csv  
  inflating: /content/data/crime-data.csv  


In [ ]:
import pandas as pd

# 🔹 1. Suç verisi (2020–2024)
crime = pd.read_csv("data/crime-data.csv", parse_dates=["Date"], low_memory=False)
crime = crime[(crime["Date"].dt.year >= 2020) & (crime["Date"].dt.year <= 2024)]
crime = crime.dropna(subset=["Community Area"])
crime["Community Area"] = crime["Community Area"].astype(int)
crime["date_only"] = crime["Date"].dt.date

daily = (
    crime
    .groupby(["date_only", "Community Area"])
    .size()
    .reset_index(name="crime_cnt")
)

# 🔹 2. Hava verisi (2020–2024)
wx = (
    pd.read_csv("data/weather-data.csv", parse_dates=["datetime"])
      .assign(date_only=lambda df: df["datetime"].dt.date)
      .groupby("date_only")
      .agg(temp=("temp", "mean"),
           humidity=("humidity", "mean"),
           precip=("precip", "sum"),
           windspeed=("windspeed", "mean"),
           visibility=("visibility", "mean"),
           uvindex=("uvindex", "mean"))
      .reset_index()
)

# 🔹 3. Nüfus verisi
pop = (
    pd.read_csv("data/chicago_population_named.csv")
      .rename(columns={"community_area_number": "Community Area"})
)

# 🔹 4. Birleştir
df = (
    daily
    .merge(wx, on="date_only", how="left")
    .merge(pop, on="Community Area", how="left")
)

print("✅ Birleştirilmiş veri:", df.shape)
df.head()


✅ Birleştirilmiş veri: (71099, 18)


,date_only,Community Area,crime_cnt,temp,humidity,precip,windspeed,visibility,uvindex,Community Name,name,population,income,latinos,blacks,white,asian,other
0,2020-01-01,1,1,32.9,71.1,0.0,19.6,9.9,4.0,Rogers Park,NaN,54991,39482,0.244,0.263,0.393,0.064,0.036
1,2020-01-01,2,3,32.9,71.1,0.0,19.6,9.9,4.0,West Ridge,NaN,71942,47323,0.204,0.111,0.427,0.225,0.032
2,2020-01-01,3,1,32.9,71.1,0.0,19.6,9.9,4.0,Uptown,NaN,56362,40324,0.142,0.200,0.516,0.114,0.028
3,2020-01-01,6,3,32.9,71.1,0.0,19.6,9.9,4.0,Lake View,NaN,94368,70746,0.076,0.039,0.804,0.060,0.021
4,2020-01-01,7,2,32.9,71.1,0.0,19.6,9.9,4.0,Lincoln Park,NaN,64116,82707,0.056,0.043,0.829,0.051,0.021


In [ ]:
!pip install astral==3.2 --quiet

In [ ]:
import numpy as np
import datetime as dt
from pandas.tseries.holiday import USFederalHolidayCalendar
from astral import LocationInfo
from astral.sun import daylight


chi = LocationInfo(41.88, -87.63)


df["crime_rate"] = df["crime_cnt"] / (df["population"] / 100_000)

df["day_name"] = pd.to_datetime(df["date_only"]).dt.day_name()
df["month"] = pd.to_datetime(df["date_only"]).dt.month

holidays = USFederalHolidayCalendar().holidays(start=df["date_only"].min(),
                                                end=df["date_only"].max())
df["is_holiday"] = df["date_only"].isin(holidays)
df["daylight_hrs"] = df["date_only"].apply(lambda d: (daylight(chi.observer, date=d)[1] - daylight(chi.observer, date=d)[0]).seconds / 3600)


df = df.sort_values(["Community Area", "date_only"])
df["lag_1"] = df.groupby("Community Area")["crime_rate"].shift(1)
df["lag_7"] = df.groupby("Community Area")["crime_rate"].shift(7)


df["lag_1"].fillna(0, inplace=True)
df["lag_7"].fillna(0, inplace=True)

print("✅ Özellikli veri:", df.shape)
df[["date_only", "Community Area", "crime_rate", "lag_1", "lag_7"]].head(10)


✅ Özellikli veri: (71099, 25)


<ipython-input-11-bf857571ae8c>:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["lag_1"].fillna(0, inplace=True)
<ipython-input-11-bf857571ae8c>:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

,date_only,Community Area,crime_rate,lag_1,lag_7
0,2020-01-01,1,1.818479,0.000000,0.000000
54,2020-01-02,1,7.273918,1.818479,0.000000
100,2020-01-03,1,1.818479,7.273918,0.000000
149,2020-01-04,1,5.455438,1.818479,0.000000
203,2020-01-05,1,1.818479,5.455438,0.000000
250,2020-01-06,1,1.818479,1.818479,0.000000
297,2020-01-07,1,1.818479,1.818479,0.000000
342,2020-01-08,1,7.273918,1.818479,1.818479
428,2020-01-10,1,1.818479,7.273918,7.273918
529,2020-01-12,1,9.092397,1.818479,1.818479


In [21]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error


le_day = LabelEncoder()
df["day_enc"] = le_day.fit_transform(df["day_name"])

feat_cols = [
    "temp", "humidity", "precip", "windspeed", "visibility", "uvindex",
    "daylight_hrs", "is_holiday", "month", "day_enc",
    "lag_1", "lag_7", "Community Area"
]
X = df[feat_cols].copy()
y = df["crime_rate"]


scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=42)
model.fit(X_train, y_train)


y_pred_fast = model.predict(X_test)
print(f"📈 R² : {r2_score(y_test, y_pred_fast):.3f}")
print(f"✅ MAE: {mean_absolute_error(y_test, y_pred_fast):.2f} suç / 100k")


NameError: name 'df' is not defined

In [ ]:

df_may = pd.read_csv("01_13 MAY.csv")


df_may["Date"] = pd.to_datetime(df_may["Date"], errors="coerce")
df_may["date_only"] = df_may["Date"].dt.date


may12 = df_may[df_may["date_only"] == dt.date(2025, 5, 12)]


real_ca18 = len(may12[may12["Community Area"] == 18])


print(f"📍 CA 18 → Gerçek: {real_ca18} | Tahmin: {round(pred_count_18, 2)}")
print(f"🧮 CA 18 Hata: {round(pred_count_18 - real_ca18, 2)}")


<ipython-input-19-c9ee11bb92b9>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_may["Date"] = pd.to_datetime(df_may["Date"], errors="coerce")


📍 CA 18 → Gerçek: 1 | Tahmin: 1.07
🧮 CA 18 Hata: 0.07


In [ ]:
# 📦 Required libraries
import pandas as pd
import numpy as np
import datetime as dt
from astral import LocationInfo
from astral.sun import daylight
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# 📁 Unzip data
!unzip -q "MelihDurhasanDSA210SprintermProject-main (1).zip"
!unzip -q "/content/project/MelihDurhasanDSA210SprintermProject-main/DSA210_datas.zip"

# 🔹 Crime data
crime = pd.read_csv("data/crime-data.csv", parse_dates=["Date"], low_memory=False)
crime = crime[(crime["Date"].dt.year >= 2020) & (crime["Date"].dt.year <= 2024)]
crime = crime.dropna(subset=["Community Area"])
crime["date_only"] = crime["Date"].dt.date

# 🔹 Daily crime counts per area
daily = crime.groupby(["date_only", "Community Area"]).size().reset_index(name="crime_cnt")

# 🔹 Weather data
weather = pd.read_csv("data/weather-data.csv", parse_dates=["datetime"])
weather["date_only"] = weather["datetime"].dt.date
wx = (weather.groupby("date_only")
      .agg(temp=("temp", "mean"),
           humidity=("humidity", "mean"),
           precip=("precip", "sum"),
           windspeed=("windspeed", "mean"),
           visibility=("visibility", "mean"),
           uvindex=("uvindex", "mean"))
      .reset_index())

# 🔹 Population data
pop = pd.read_csv("data/chicago_population_named.csv")

# 🔹 Merge datasets
df = (daily.merge(wx, on="date_only", how="left")
          .merge(pop[["Community Area", "population"]], on="Community Area", how="left"))

# 🔹 Feature engineering
chi = LocationInfo(41.88, -87.63)
df["crime_rate"] = df["crime_cnt"] / (df["population"] / 100_000)
df["day_name"] = pd.to_datetime(df["date_only"]).dt.day_name()
df["month"] = pd.to_datetime(df["date_only"]).dt.month
holidays = USFederalHolidayCalendar().holidays(start=df["date_only"].min(), end=df["date_only"].max())
df["is_holiday"] = df["date_only"].isin(holidays)
df["daylight_hrs"] = df["date_only"].apply(lambda d: (daylight(chi.observer, date=d)[1] - daylight(chi.observer, date=d)[0]).seconds / 3600)

# 🔹 Lag features
df = df.sort_values(["Community Area", "date_only"])
df["lag_1"] = df.groupby("Community Area")["crime_rate"].shift(1)
df["lag_7"] = df.groupby("Community Area")["crime_rate"].shift(7)
df["lag_1"].fillna(0, inplace=True)
df["lag_7"].fillna(0, inplace=True)

# 🔹 Encoding & model inputs
le_day = LabelEncoder()
df["day_enc"] = le_day.fit_transform(df["day_name"])
feat_cols = ["temp", "humidity", "precip", "windspeed", "visibility", "uvindex",
             "daylight_hrs", "is_holiday", "month", "day_enc", "lag_1", "lag_7", "Community Area"]
X = df[feat_cols]
y = df["crime_rate"]
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# 🔹 Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=42)
model.fit(X_train, y_train)

# 🔹 Evaluation
y_pred = model.predict(X_test)
print(f"📈 R² score : {r2_score(y_test, y_pred):.3f}")
print(f"✅ MAE       : {mean_absolute_error(y_test, y_pred):.2f} crimes per 100k people")

# 🔹 Weather on May 12, 2025 from VisualCrossing
weather_12_may = {
    "temp": 60,
    "humidity": 79.4,
    "precip": 0.285,
    "windspeed": 22.9,
    "visibility": 8.6,
    "uvindex": 2
}

def predict_crime(date_str, community_area, **weather):
    d = dt.datetime.strptime(date_str, "%Y-%m-%d").date()
    daylight_hrs = (daylight(chi.observer, date=d)[1] - daylight(chi.observer, date=d)[0]).seconds / 3600
    try:
        lag_1 = df.loc[(df["date_only"] == d - dt.timedelta(days=1)) & (df["Community Area"] == community_area), "crime_rate"].values[0]
    except:
        lag_1 = 0
    try:
        lag_7 = df.loc[(df["date_only"] == d - dt.timedelta(days=7)) & (df["Community Area"] == community_area), "crime_rate"].values[0]
    except:
        lag_7 = 0
    row = {
        "temp": weather.get("temp", 60),
        "humidity": weather.get("humidity", 50),
        "precip": weather.get("precip", 0.0),
        "windspeed": weather.get("windspeed", 10),
        "visibility": weather.get("visibility", 10),
        "uvindex": weather.get("uvindex", 4),
        "daylight_hrs": daylight_hrs,
        "is_holiday": int(d in holidays),
        "month": d.month,
        "day_enc": le_day.transform([d.strftime("%A")])[0],
        "lag_1": lag_1,
        "lag_7": lag_7,
        "Community Area": community_area
    }
    X_input = pd.DataFrame([row], columns=feat_cols)
    X_scaled_input = scaler.transform(X_input)
    return model.predict(X_scaled_input)[0]

# 🔹 Prediction for CA 18 on May 12, 2025
pred_rate_18 = predict_crime("2025-05-12", 18, **weather_12_may)
pop18 = df.loc[df["Community Area"] == 18, "population"].iloc[0]
pred_count_18 = pred_rate_18 * pop18 / 100_000

print(f"\n📍 CA 18 → Predicted crime rate: {pred_rate_18:.2f} /100k")
print(f"🤖 Predicted number of crimes : {pred_count_18:.2f}")

# 🔹 Compare with real data from 01_13 MAY.csv
df_may = pd.read_csv("01_13 MAY.csv")
df_may["Date"] = pd.to_datetime(df_may["Date"], errors="coerce")
df_may["date_only"] = df_may["Date"].dt.date
may12 = df_may[df_may["date_only"] == dt.date(2025, 5, 12)]
real_ca18 = len(may12[may12["Community Area"] == 18])

print(f"\n📌 Real crime count (CA 18): {real_ca18}")
print(f"🧮 Prediction error         : {round(pred_count_18 - real_ca18, 2)}")


<ipython-input-23-60fdaf0a97d3>:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["lag_1"].fillna(0, inplace=True)
<ipython-input-23-60fdaf0a97d3>:59: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

📈 R² score : 0.574
✅ MAE       : 2.76 crimes per 100k people

📍 CA 18 → Predicted crime rate: 7.95 /100k
🤖 Predicted number of crimes : 1.07


<ipython-input-23-60fdaf0a97d3>:131: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_may["Date"] = pd.to_datetime(df_may["Date"], errors="coerce")



📌 Real crime count (CA 18): 1
🧮 Prediction error         : 0.07
